In [487]:
# Importo las librerías que voy a usar.

import pymongo
from pymongo import MongoClient
client = MongoClient ('localhost', 27017)
db = client['companies']

from pandas.io.json import json_normalize

import pandas as pd

In [488]:
# Obtengo todo el dataset para visualizar bien los datos.

data = db.companies.find()
dataset = pd.DataFrame(list(data))
pd.set_option("display.max_columns", 500)
display(dataset.head())

,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,deadpooled_day,deadpooled_month,deadpooled_url,deadpooled_year,description,email_address,external_links,founded_day,founded_month,founded_year,funding_rounds,homepage_url,image,investments,ipo,milestones,name,number_of_employees,offices,overview,partners,permalink,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297d8a,"{'price_amount': 30000000, 'price_currency_cod...",[],,http://digitalquarters.net/feed/,http://digitalquarters.net/,web,"[{'competitor': {'name': 'Wikia', 'permalink':...",2007-05-25 06:51:27,http://www.crunchbase.com/company/wetpaint,NaN,NaN,NaN,1.0,Technology Platform Company,info@wetpaint.com,[{'external_url': 'http://www.geekwire.com/201...,17.0,10.0,2005.0,"[{'id': 888, 'round_code': 'a', 'source_url': ...",http://wetpaint-inc.com,"{'available_sizes': [[[150, 75], 'assets/image...",[],NaN,"[{'id': 5869, 'description': 'Wetpaint named i...",Wetpaint,47.0,"[{'description': '', 'address1': '710 - 2nd Av...",<p>Wetpaint is a technology platform company t...,[],abc2,206.859.6300,"[{'name': 'Wikison Wetpaint', 'permalink': 'we...",[],"[{'is_past': False, 'title': 'Co-Founder and V...","[{'available_sizes': [[[150, 86], 'assets/imag...","wiki, seattle, elowitz, media-industry, media-...",$39.8M,BachelrWetpaint,Sun Dec 08 07:15:44 UTC 2013,[]
1,52cdef7c4bab8bd675297d8b,None,[],Zoho ManageEngine,,,enterprise,[],2007-05-25 19:24:22,http://www.crunchbase.com/company/adventnet,NaN,NaN,NaN,2.0,Server Management Software,pr@adventnet.com,[],NaN,NaN,1996.0,[],http://adventnet.com,"{'available_sizes': [[[150, 55], 'assets/image...",[],NaN,[],AdventNet,600.0,"[{'description': 'Headquarters', 'address1': '...","<p>AdventNet is now <a href=""/company/zoho-man...",[],abc3,925-924-9500,[],"[{'title': 'DHFH', 'is_past': True, 'provider'...","[{'is_past': True, 'title': 'CEO and Co-Founde...","[{'available_sizes': [[[150, 94], 'assets/imag...",,$0,manageengine,Wed Oct 31 18:26:09 UTC 2012,[]
2,52cdef7c4bab8bd675297d8c,None,[],,http://blogs.zoho.com/feed,http://blogs.zoho.com/,software,"[{'competitor': {'name': 'Empressr', 'permalin...",Fri May 25 19:30:28 UTC 2007,http://www.crunchbase.com/company/zoho,NaN,NaN,NaN,3.0,Online Business Apps Suite,info@zohocorp.com,[{'external_url': 'http://www.online-tech-tips...,15.0,9.0,2005.0,[],http://zoho.com,"{'available_sizes': [[[150, 55], 'assets/image...",[],NaN,"[{'id': 388, 'description': 'Zoho Reaches 2 Mi...",Zoho,1600.0,"[{'description': 'Headquarters', 'address1': '...","<p>Zoho offers a suite of Business, Collaborat...",[],abc4,1-888-204-3539,"[{'name': 'Zoho Office Suite', 'permalink': 'z...",[],"[{'is_past': False, 'title': 'CEO and Founder'...",[],"zoho, officesuite, spreadsheet, writer, projec...",$0,zoho,Wed Oct 30 00:07:05 UTC 2013,"[{'embed_code': '<object width=""430"" height=""2..."
3,52cdef7c4bab8bd675297d8d,"{'price_amount': 500000, 'price_currency_code'...","[{'price_amount': None, 'price_currency_code':...",,http://blog.digg.com/?feed=rss2,http://blog.digg.com/,news,"[{'competitor': {'name': 'Reddit', 'permalink'...",Fri May 25 20:03:23 UTC 2007,http://www.crunchbase.com/company/digg,NaN,NaN,None,NaN,user driven social content website,feedback@digg.com,[{'external_url': 'http://www.sociableblog.com...,11.0,10.0,2004.0,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...",http://www.digg.com,"{'available_sizes': [[[150, 150], 'assets/imag...",[],None,"[{'id': 9588, 'description': 'Another Digg Exe...",Digg,60.0,"[{'description': None, 'address1': '135 Missis...",<p>Digg is a user driven social content websit...,[],digg,(415) 436-9638,"[{'name': 'Digg', 'permalink': 'digg'}]","[{'title': 'Public Relations', 'is_past': True...","[{'is_past': False, 'title': 'CEO', 'person': ...","[{'available_sizes': [[[117, 150], 'assets/ima...","community, social, news, bo

In [489]:
# Hago de nuevo la query, quedándome con lo que me interesa.

mydataset = db.companies.find({"$and": 
                               [{"offices.latitude": {"$exists": True}}, {"offices.latitude": {"$ne": None}},
                                {"offices.longitude": {"$exists": True}}, {"offices.longitude": {"$ne": None}},
                                {"category_code": {"$exists": True}}, {"category_code": {"$ne": None}},
                                {"name": {"$exists": True}}, {"name": {"$ne": None}},
                                {"number_of_employees": {"$exists": True}}, {"number_of_employees": {"$ne": None}},
                                {"founded_year": {"$exists": True}}, {"founded_year": {"$gte": 2008}},
                                {"deadpooled_year": None},
                                {"category_code": {"$ne": "other"}}, {"category_code": {"$ne": "semiconductor"}}, 
                                {"category_code": {"$ne": "finance"}}, {"category_code": {"$ne": "medical"}}, 
                                {"category_code": {"$ne": "health"}}, {"category_code": {"$ne": "manufacturing"}}, 
                                {"category_code": {"$ne": "real_estate"}}, {"category_code": {"$ne": "fashion"}}, 
                                {"category_code": {"$ne": "nanotech"}}, {"category_code": {"$ne": "hospitality"}}, 
                                {"category_code": {"$ne": "transportation"}}, {"category_code": {"$ne": "local"}}]}, 
                              {"_id":1, "category_code":1,"founded_year":1, "name":1, 
                               "number_of_employees":1, "offices.country_code":1, "offices.latitude":1, 
                               "offices.longitude":1, "total_money_raised":1})

In [490]:
# Normalizo la columna offices, para separar los subvalores en columnas.

mydataframe = json_normalize(data = mydataset, record_path = "offices", 
                             meta = ["_id", "category_code", "founded_year", "name",
                                    "number_of_employees", "total_money_raised"])

In [491]:
mydataframe.head(10)

,country_code,latitude,longitude,_id,category_code,founded_year,name,number_of_employees,total_money_raised
0,USA,37.782263,-122.392142,52cdef7c4bab8bd675297e30,software,2008,GoingOn,40,$8.5M
1,USA,42.358920,-71.057810,52cdef7c4bab8bd675297ee1,enterprise,2008,Cloudant,70,$18.3M
2,USA,47.597965,-122.151158,52cdef7c4bab8bd675297f3f,games_video,2008,YouBeQB,8,$0
3,USA,37.781265,-122.393229,52cdef7c4bab8bd675297f76,games_video,2008,Crunchyroll,50,$4.8M
4,USA,40.757929,-73.985506,52cdef7c4bab8bd675297f94,search,2012,PeekYou,20,$1.83M
5,USA,33.817100,-111.903500,52cdef7c4bab8bd675297f9e,mobile,2010,GENWI,25,$7.1M
6,USA,37.566879,-122.323895,52cdef7c4bab8bd675297fec,web,2013,Fixya,30,$8M
7,DEU,53.707739,10.023246,52cdef7c4bab8bd675298038,games_video,2009,alluc,7,$0
8,CZE,49.225902,16.569282,52cdef7c4bab8bd6752981e8,network_hosting,2008,Webnode,30,$0
9,USA,34.031764,-118.461884,52cdef7c4bab8bd675298218,games_video,2008,WonderHowTo,12,$0


In [492]:
# Creo una columna que una las coordenadas, porque creo que la necesitaré para indexarla después en compass. Me han
# dicho que compass interpreta las coordenadas como [long, lat], en lugar de [lat, long]. Creo columnas con ambas 
# opciones por si acaso.

from geojson import Point

# Creo una lista uniendo las latitudes y longitudes (Una para cada orden por si acaso).

coordinates_longlat = []

for i in range(len(mydataframe["latitude"])):
    coordinates_longlat.append((mydataframe["longitude"][i], mydataframe["latitude"][i]))
    
coordinates_latlong = []

for i in range(len(mydataframe["latitude"])):
    coordinates_latlong.append((mydataframe["latitude"][i], mydataframe["longitude"][i]))

In [493]:
# Actualizo las listas convirtiendo sus valores a puntos geoespaciales (no me dejaba hacerlo directamente arriba).
                               
coordinates_longlat2 = []

for i in coordinates_longlat:
    coordinates_longlat2.append(Point(i))
    
coordinates_latlong2 = []

for i in coordinates_latlong:
    coordinates_latlong2.append(Point(i))

In [494]:
# Creo columnas con los valores de las coordenadas en formato geoespacial.

mydataframe["coordenadas"] = coordinates_longlat
mydataframe["coordinates_longlat"] = coordinates_longlat2
mydataframe["coordinates_latlong"] = coordinates_latlong2

In [495]:
mydataframe.head()

,country_code,latitude,longitude,_id,category_code,founded_year,name,number_of_employees,total_money_raised,coordenadas,coordinates_longlat,coordinates_latlong
0,USA,37.782263,-122.392142,52cdef7c4bab8bd675297e30,software,2008,GoingOn,40,$8.5M,"(-122.392142, 37.782263)","{'type': 'Point', 'coordinates': [-122.392142,...","{'type': 'Point', 'coordinates': [37.782263, -..."
1,USA,42.358920,-71.057810,52cdef7c4bab8bd675297ee1,enterprise,2008,Cloudant,70,$18.3M,"(-71.05781, 42.35892)","{'type': 'Point', 'coordinates': [-71.05781, 4...","{'type': 'Point', 'coordinates': [42.35892, -7..."
2,USA,47.597965,-122.151158,52cdef7c4bab8bd675297f3f,games_video,2008,YouBeQB,8,$0,"(-122.151158, 47.597965)","{'type': 'Point', 'coordinates': [-122.151158,...","{'type': 'Point', 'coordinates': [47.597965, -..."
3,USA,37.781265,-122.393229,52cdef7c4bab8bd675297f76,games_video,2008,Crunchyroll,50,$4.8M,"(-122.393229, 37.781265)","{'type': 'Point', 'coordinates': [-122.393229,...","{'type': 'Point', 'coordinates': [37.781265, -..."
4,USA,40.757929,-73.985506,52cdef7c4bab8bd675297f94,search,2012,PeekYou,20,$1.83M,"(-73.985506, 40.757929)","{'type': 'Point', 'coordinates': [-73.985506, ...","{'type': 'Point', 'coordinates': [40.757929, -..."


In [496]:
# Reordeno mis columnas para una mejor visualización.

columns = ["_id", "name", "category_code", "country_code", "founded_year", "number_of_employees", "total_money_raised",
           "coordenadas", "coordinates_longlat", "coordinates_latlong", "latitude", "longitude"]

mydataframe = mydataframe.reindex(columns, axis= 1)

mydataframe.head()

,_id,name,category_code,country_code,founded_year,number_of_employees,total_money_raised,coordenadas,coordinates_longlat,coordinates_latlong,latitude,longitude
0,52cdef7c4bab8bd675297e30,GoingOn,software,USA,2008,40,$8.5M,"(-122.392142, 37.782263)","{'type': 'Point', 'coordinates': [-122.392142,...","{'type': 'Point', 'coordinates': [37.782263, -...",37.782263,-122.392142
1,52cdef7c4bab8bd675297ee1,Cloudant,enterprise,USA,2008,70,$18.3M,"(-71.05781, 42.35892)","{'type': 'Point', 'coordinates': [-71.05781, 4...","{'type': 'Point', 'coordinates': [42.35892, -7...",42.358920,-71.057810
2,52cdef7c4bab8bd675297f3f,YouBeQB,games_video,USA,2008,8,$0,"(-122.151158, 47.597965)","{'type': 'Point', 'coordinates': [-122.151158,...","{'type': 'Point', 'coordinates': [47.597965, -...",47.597965,-122.151158
3,52cdef7c4bab8bd675297f76,Crunchyroll,games_video,USA,2008,50,$4.8M,"(-122.393229, 37.781265)","{'type': 'Point', 'coordinates': [-122.393229,...","{'type': 'Point', 'coordinates': [37.781265, -...",37.781265,-122.393229
4,52cdef7c4bab8bd675297f94,PeekYou,search,USA,2012,20,$1.83M,"(-73.985506, 40.757929)","{'type': 'Point', 'coordinates': [-73.985506, ...","{'type': 'Point', 'coordinates': [40.757929, -...",40.757929,-73.985506


In [497]:
null_cols = mydataframe.isnull().sum()

null_cols[null_cols > 0]

Series([], dtype: int64)

In [498]:
# Categorizo en bins los valores de number_of_employees.

enterprize_size = ["small", "medium", "big"]
cutoffs = [-1, 10, 99, float("inf")]

bins = pd.cut(mydataframe["number_of_employees"], cutoffs, labels = enterprize_size)
mydataframe["number_of_employees"] = bins

In [502]:
mydataframe["category_code"].value_counts()

web                 365
software            142
games_video          94
advertising          90
mobile               79
ecommerce            78
consulting           74
enterprise           69
public_relations     49
search               34
network_hosting      32
social               14
analytics            10
security             10
hardware              9
cleantech             9
education             7
messaging             6
music                 5
design                4
news                  4
biotech               4
travel                4
legal                 2
sports                2
photo_video           1
Name: category_code, dtype: int64

In [503]:
# Categorizo en bins los valores de category_code.

sector_interes = {"web": 8, "software": 9, "games_video": 10, "advertising": 8, "mobile": 7, "ecommerce": 9, 
                    "consulting": 6, "enterprise": 5, "public_relations": 5, "search": 5, "network_hosting": 5, 
                    "social": 5, "analytics": 9, "security": 6, "hardware": 9, "cleantech": 8, "education": 5, 
                    "messaging": 8, "design": 9, "news": 8, "biotech": 9, "travel": 5, "legal": 8, "sports": 7, 
                    "photo_video": 7}

In [499]:
mydataframe.head()

,_id,name,category_code,country_code,founded_year,number_of_employees,total_money_raised,coordenadas,coordinates_longlat,coordinates_latlong,latitude,longitude
0,52cdef7c4bab8bd675297e30,GoingOn,software,USA,2008,medium,$8.5M,"(-122.392142, 37.782263)","{'type': 'Point', 'coordinates': [-122.392142,...","{'type': 'Point', 'coordinates': [37.782263, -...",37.782263,-122.392142
1,52cdef7c4bab8bd675297ee1,Cloudant,enterprise,USA,2008,medium,$18.3M,"(-71.05781, 42.35892)","{'type': 'Point', 'coordinates': [-71.05781, 4...","{'type': 'Point', 'coordinates': [42.35892, -7...",42.358920,-71.057810
2,52cdef7c4bab8bd675297f3f,YouBeQB,games_video,USA,2008,small,$0,"(-122.151158, 47.597965)","{'type': 'Point', 'coordinates': [-122.151158,...","{'type': 'Point', 'coordinates': [47.597965, -...",47.597965,-122.151158
3,52cdef7c4bab8bd675297f76,Crunchyroll,games_video,USA,2008,medium,$4.8M,"(-122.393229, 37.781265)","{'type': 'Point', 'coordinates': [-122.393229,...","{'type': 'Point', 'coordinates': [37.781265, -...",37.781265,-122.393229
4,52cdef7c4bab8bd675297f94,PeekYou,search,USA,2012,medium,$1.83M,"(-73.985506, 40.757929)","{'type': 'Point', 'coordinates': [-73.985506, ...","{'type': 'Point', 'coordinates': [40.757929, -...",40.757929,-73.985506


In [500]:
# No me dejaba exportar a json porque había un caracter raro en mi dataframe. He probado a eliminar la columna _id,
# y ya me deja.

mydataframe = mydataframe.drop("_id", axis = 1)

In [501]:
# Exporto mi dataframe a json.

mydataframe.to_json("geospatial_clean.json", orient = "records", lines = True)